# **BasketBall Playoffs**

### CSV information loaded into Dataframes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC


In [2]:
awards_players_df = pd.read_csv("data/awards_players.csv")
coaches_df = pd.read_csv("data/coaches.csv")
players_teams_df = pd.read_csv("data/players_teams.csv")
players_df = pd.read_csv("data/players.csv")
series_post_df = pd.read_csv("data/series_post.csv")
teams_post_df = pd.read_csv("data/teams_post.csv")
teams_df = pd.read_csv("data/teams.csv")
players_df.isna().all()

bioID           False
pos             False
firstseason     False
lastseason      False
height          False
weight          False
college         False
collegeOther    False
birthDate       False
deathDate       False
dtype: bool

### Dropping some columns and rows
Some invalid player entries were found with null values in every column.

We also dropped all columns relative to *lgID* as all rows had the same value in every dataframe


In [3]:
#death date isnt relevant and its value is always 0000-00-00
players_df = players_df.drop(["deathDate","firstseason","lastseason"], axis="columns")
#lgID(Winner/Loser) isnt relevant as its value is always the same
awards_players_df = awards_players_df.drop(["lgID"], axis="columns")
coaches_df = coaches_df.drop(["lgID"], axis="columns")
players_teams_df = players_teams_df.drop(["lgID"], axis="columns")
series_post_df = series_post_df.drop(["lgIDWinner","lgIDLoser"], axis="columns")
teams_post_df = teams_post_df.drop(["lgID"], axis="columns")
teams_df = teams_df.drop(["lgID","seeded","arena","franchID","name","divID"], axis="columns")
#remove players whose birthDate is 0000-00-00
players_df = players_df[players_df['birthDate'] != '0000-00-00']
players_df = players_df.rename(columns={"bioID": "playerID"})


In [4]:
merged_df = pd.merge(players_teams_df, players_df, how="left", on=["playerID"])
merged_df = pd.merge(merged_df, awards_players_df, how="left", on=["playerID","year"])
teams_merged_df = pd.merge(teams_df, coaches_df, how="left", on=["year","tmID"])
merged_df = pd.merge(merged_df, teams_merged_df, how="left", on=["year","tmID"])
merged_df["award"] = merged_df["award"].fillna("None")
merged_df["college"] = merged_df["college"].fillna("None")
merged_df["collegeOther"] = merged_df["collegeOther"].fillna("None")
merged_df["firstRound"] = merged_df["firstRound"].fillna("None")
merged_df["semis"] = merged_df["semis"].fillna("None")
merged_df["finals"] = merged_df["finals"].fillna("None")            



In [5]:

def calculate_age(merged_df):

    if not pd.api.types.is_datetime64_dtype(merged_df['birthDate']):
        merged_df['birthDate'] = pd.to_datetime(merged_df['birthDate'])
    
    merged_df['age'] = 2014 + merged_df['year'] - merged_df['birthDate'].dt.year
    

    merged_df.drop(columns=['birthDate'],inplace=True)

    return merged_df
    
calculate_age(merged_df)
merged_df
    

,playerID,year,stint_x,tmID,GP_x,GS,minutes,points,oRebounds,dRebounds,...,confL,min,attend,coachID,stint_y,won_y,lost_y,post_wins,post_losses,age
0,abrossv01w,2,0,MIN,26,23,846,343,43,131,...,12,6475,120607,aglerbr99w,0,12,20,0,0,36
1,abrossv01w,3,0,MIN,27,27,805,314,45,101,...,15,6475,139874,aglerbr99w,1,6,13,0,0,37
2,abrossv01w,3,0,MIN,27,27,805,314,45,101,...,15,6475,139874,vandehe99w,2,4,9,0,0,37
3,abrossv01w,4,0,MIN,30,25,792,318,44,97,...,10,6850,120253,mcconsu01w,0,18,16,1,2,38
4,abrossv01w,5,0,MIN,22,11,462,146,17,57,...,10,6850,125097,mcconsu01w,0,18,16,0,2,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,zarafr01w,6,0,SEA,34,4,413,90,11,28,...,9,6825,151150,donovan99w,0,20,14,1,2,44
2156,zellosh01w,10,0,DET,34,4,802,406,25,81,...,11,6975,136069,laimbbi01w,1,1,2,0,0,38
2157,zellosh01w,10,0,DET,34,4,802,406,25,81,...,11,6975,136069,mahorri01w,2,17,14,3,2,38
2158,zirkozu01w,4,0,WAS,6,0,30,11,0,2,...,17,6875,238710,stanlma99w,0,9,25,0,0,38


In [6]:
encoder = LabelEncoder()
merged_df["playoff"] = encoder.fit_transform(merged_df["playoff"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

N: 0
Y: 1


In [7]:
merged_df["firstRound"] = encoder.fit_transform(merged_df["firstRound"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

L: 0
None: 1
W: 2


In [8]:
merged_df["semis"] = encoder.fit_transform(merged_df["semis"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

L: 0
None: 1
W: 2


In [9]:
merged_df["finals"] = encoder.fit_transform(merged_df["finals"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

L: 0
None: 1
W: 2


In [10]:
merged_df["pos"] = encoder.fit_transform(merged_df["pos"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

C: 0
C-F: 1
F: 2
F-C: 3
F-G: 4
G: 5
G-F: 6


In [11]:
merged_df["confID"] = encoder.fit_transform(merged_df["confID"])

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    print(f"{label}: {idx}")

EA: 0
WE: 1


In [12]:
merged_df["college"] = encoder.fit_transform(merged_df["college"])

college_normalization_rules = {
    "NE Oklahoma A&M (JC)":126,
    "Arkansas-Fort Smith (fr-so)":127,
    "Grayson County (JC)":128
}

# print what was mapped to what
for idx, label in enumerate(encoder.classes_):
    college_normalization_rules[label] = idx

    print(f"{label}: {college_normalization_rules[label]}")

Academy of Sport Moscow: 0
Alabama: 1
Alabama-Birmingham: 2
Arizona: 3
Arizona State: 4
Arkansas: 5
Arkansas State: 6
Auburn: 7
Australian Institute of Sport: 8
Baylor: 9
Boise State: 10
Boston College: 11
Brigham Young: 12
California: 13
Central Florida: 14
Cincinnati: 15
Clemson: 16
Colorado: 17
Colorado State: 18
Connecticut: 19
DePaul: 20
Delaware: 21
Denver: 22
Duke: 23
Duquesne: 24
Flordia: 25
Florida: 26
Florida Atlantic: 27
Florida International: 28
Florida State: 29
Furman: 30
George Mason: 31
George Washington: 32
Georgetown: 33
Georgia: 34
Georgia Tech: 35
Harvard: 36
Hawai'i: 37
Houston: 38
Howard: 39
Illinois: 40
Illinois State: 41
Indiana: 42
Iona: 43
Iowa: 44
Iowa State: 45
James Madison: 46
Kansas: 47
Kansas State: 48
Kentucky: 49
Liberty: 50
Long Beach State: 51
Louisiana State: 52
Louisiana Tech: 53
Louisville: 54
Maine: 55
Maryland: 56
Memphis: 57
Miami (FL): 58
Michigan: 59
Michigan State: 60
Middle Tennessee State: 61
Minnesota: 62
Mississippi: 63
Mississippi State

In [13]:
def normalize_collegeOther(name):
    return college_normalization_rules[name]

merged_df["collegeO"] = merged_df["collegeOther"].apply(normalize_collegeOther)
merged_df = merged_df.drop(["collegeOther"], axis="columns")

In [14]:
def normalize_awards(award):
    normalization_rules = {
        "None": 0,
        "All-Star Game Most Valuable Player": 1,
        "Coach of the Year": 2,
        "Defensive Player of the Year": 3,
        "Kim Perrot Sportsmanship": 4,
        "Kim Perrot Sportsmanship Award": 4,
        "Most Improved Player":5,
        "Most Valuable Player":6,
        "Rookie of the Year":7,
        "Sixth Woman of the Year":8,
        "WNBA Finals Most Valuable Player":9,
        "WNBA All-Decade Team":10,
        "WNBA All Decade Team Honorable Mention":11
    }
    return normalization_rules[award]

merged_df["normalized_award"] = merged_df["award"].apply(normalize_awards)
merged_df = merged_df.drop(["award"], axis="columns")


In [15]:
import pandas as pd

# Assuming 'merged_df' is your DataFrame
nan_counts = merged_df.isnull().sum()

# Filter to show only columns with NaN values
nan_columns = nan_counts[nan_counts > 0]

print("Columns with NaN values:")
print(nan_columns)

Columns with NaN values:
Series([], dtype: int64)


In [16]:
non_numeric_columns = merged_df.select_dtypes(exclude=['number']).columns.tolist()

print("Non-numeric columns:")
print(non_numeric_columns)



Non-numeric columns:
['playerID', 'tmID', 'coachID']


In [17]:
plt.figure(figsize=(25,25))
plt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.4,hspace=0.4)

merged_df1 = merged_df[merged_df['playoff']==1].drop(['playoff'],axis=1)
merged_df2 = merged_df[merged_df['playoff']==0].drop(['playoff'],axis=1)

cols = list(merged_df1.columns)

for i, col in enumerate(cols):
    plt.subplot(11,11,i+1)
    sb.histplot(data=merged_df1[col], color='red', label = 'Y', kde=True)
    sb.histplot(data=merged_df2[col], color='blue', label = 'N', kde=True)
    _,axes = plt.gca().get_legend_handles_labels()
    plt.legend(axes,prop={'size': 10})

plt.show()

"\nplt.figure(figsize=(25,25))\nplt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.4,hspace=0.4)\n\nmerged_df1 = merged_df[merged_df['playoff']==1].drop(['playoff'],axis=1)\nmerged_df2 = merged_df[merged_df['playoff']==0].drop(['playoff'],axis=1)\n\ncols = list(merged_df1.columns)\n\nfor i, col in enumerate(cols):\n    plt.subplot(11,11,i+1)\n    sb.histplot(data=merged_df1[col], color='red', label = 'Y', kde=True)\n    sb.histplot(data=merged_df2[col], color='blue', label = 'N', kde=True)\n    _,axes = plt.gca().get_legend_handles_labels()\n    plt.legend(axes,prop={'size': 10})\n\nplt.show()\n"

### The SVM Model

In [22]:
train_data = merged_df[merged_df['year'] <= 5]


X_train = train_data.drop(columns=['playoff', 'year', 'playerID', 'tmID', 'coachID'])
y_train = train_data['playoff']

svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)


X_all = merged_df.drop(columns=['playoff', 'year', 'playerID', 'tmID', 'coachID'])
predicted_probabilities = svm_model.predict_proba(X_all)[:, 1]


results_df = merged_df[['year', 'playerID', 'tmID', 'coachID']].copy()
results_df['predicted_probability'] = predicted_probabilities


results_df = results_df[merged_df['year'] <= 5]

results_df



      year    playerID tmID     coachID  predicted_probability
0        2  abrossv01w  MIN  aglerbr99w               0.434038
1        3  abrossv01w  MIN  aglerbr99w               0.515008
2        3  abrossv01w  MIN  vandehe99w               0.515008
3        4  abrossv01w  MIN  mcconsu01w               0.433284
4        5  abrossv01w  MIN  mcconsu01w               0.452094
...    ...         ...  ...         ...                    ...
2151     3  zakalok01w  DET  laimbbi01w               0.351598
2152     3  zakalok01w  DET  willigr99w               0.351598
2153     3  zakalok01w  PHO  coopecy01w               0.514239
2154     3  zakalok01w  PHO  sharpli99w               0.514238
2158     4  zirkozu01w  WAS  stanlma99w               0.703455

[1143 rows x 5 columns]
